# fastai课程笔记 ml lesson1

python小白，自学机器学习，在网络上找了一圈资料后，确定通过fastai入手开始学习。

这个课程强调doing，先上手操作找感觉。 “可以先学会开车，而不用了解引擎的工作原理”

课程路径：https://course18.fast.ai/ml.html

代码： git clone https://github.com/fastai/fastai1.git

Markdown 用法备忘

*我是斜体*   

**我加粗了**  

~~我带删除线~~  

> 在开头加 >和一个空格，表示代码的引用 

<code>#表示代码段
print('hello World')
</code>


## 准备工作

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

这几句啥意思？

参考ipython的文档：
https://ipython.readthedocs.io/en/stable/interactive/magics.html

%load_ext ：Load an IPython extension by its module name.

autoreload 意思是自动重新装入。个人理解：在修改了函数代码之后，自动重新加载。它后面可带参数。一般说：

无参：装入所有模块。
0：不执行 装入命令。
1： 只装入所有 %aimport 要装模块
2：装入所有 %aimport 不包含的模块。

%matplotlib Set up matplotlib to work interactively. 具体参考上面的文档

In [2]:
import sys
sys.path.insert(0, "F:\\fastai\\fastai1\\old\\") #windows环境，需要使用双反斜杠

这个课程实例代码，使用的是老版本的fastai，要安装fastai0.7版本。

参考 https://forums.fast.ai/t/fastai-v0-install-issues-thread/24652

通过增加syspath，使用老版本

In [3]:
from fastai.imports import *
from fastai0.structured import * #代码old目录中有老版本fastai，重命名了文件夹。

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics

C:\Users\kongxiangyu\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)


In [19]:
print(sys.modules['fastai0'])  #显示module是从哪里import

<module 'fastai0' from 'F:\\fastai\\fastai1\\old\\fastai0\\__init__.py'>


## 数据准备和简单分析

In [4]:
PATH = "data\\bulldozers\\"

这里指定了数据集的路径。

我们使用的是 kaggle里的 "Blue Book for Bulldozers" 这个compitition的数据集。
需要自行下载和解压到对应路径

https://www.kaggle.com/c/bluebook-for-bulldozers/data

PS:kaggle注册需要梯子，否则不能证明你不是机器人

In [9]:
!ls {PATH}

'ls' 不是内部或外部命令，也不是可运行的程序
或批处理文件。


!command是调用系统命令，我这里用的是windows系统，没这个命令

> 另外在vscode中，中文回显是乱码，还没解决。。。

In [13]:
!dir {PATH}

 驱动器 F 中的卷是 Data
 卷的序列号是 06B6-9A0D

 F:\fastai\fastai1\courses\ml1\data\bulldozers 的目录

2021-04-22  10:03    <DIR>          .
2021-04-22  10:03    <DIR>          ..
2021-04-21  14:09        50,792,941 bluebook-for-bulldozers.zip
2019-12-11  04:05            11,063 Data Dictionary.xlsx
2019-12-11  04:05        51,498,702 Machine_Appendix.csv
2019-12-11  04:05           196,760 median_benchmark.csv
2019-12-11  04:05           211,941 random_forest_benchmark_test.csv
2019-12-11  04:05         3,560,907 Test.csv
2019-12-11  04:05         7,183,499 Train.7z
2013-01-24  17:08       116,403,970 Train.csv
2019-12-11  04:05         9,732,240 Train.zip
2019-12-11  04:05         7,403,772 TrainAndValid.7z
2019-12-11  04:05       119,791,159 TrainAndValid.csv
2019-12-11  04:05        10,057,248 TrainAndValid.zip
2019-12-11  04:05           214,480 Valid.7z
2019-12-11  04:05         3,318,969 Valid.csv
2019-12-11  04:05           303,660 Valid.zip
2019-12-11  04:05           323,524 ValidSolution.c

Tips：快速了解某个方法是干啥的。  在notebook中也可以用 shift+tab / shift+tab*2 / shift+Tab*3

In [1]:
display

<function IPython.core.display.display(*objs, include=None, exclude=None, metadata=None, transient=None, display_id=None, **kwargs)>

In [4]:
?display   #显示函数定义和说明

In [3]:
??display  #显示函数源代码

pandas怎么用，网上可以搜到大把。
本来想花点时间系统学习一下，但是后来意识到，学完了不用，几天就忘了。
先跟着课程走下去，后面实际应用中有什么需求，再详细了解吧。

https://pandas.pydata.org/docs/user_guide/index.html#user-guide

In [81]:
df_raw = pd.read_csv(f'{PATH}Train.csv', low_memory=False, parse_dates=["saledate"])

f-string参考：

https://docs.python.org/zh-cn/3/reference/lexical_analysis.html#f-strings

https://www.python.org/dev/peps/pep-0498/

In [38]:
df_raw.saledate.head()

0   2006-11-16
1   2004-03-26
2   2004-02-26
3   2011-05-19
4   2009-07-23
Name: saledate, dtype: datetime64[ns]

注意 回显中的dtype是datetime，这里体现了read_csv中parse_dates参数的作用。如果不使用这个参数，saledate测数据格式是ojbect

In [82]:
df_raw.shape 

(401125, 53)

In [30]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)

In [41]:
#为什么要写display_all这个函数? 先看看直接display是什么样的
display(df_raw)

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1139246,66000,999089,3157,121,3.0,2004,68.0,Low,2006-11-16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
1,1139248,57000,117657,77,121,3.0,1996,4640.0,Low,2004-03-26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
2,1139249,10000,434808,7009,121,3.0,2001,2838.0,High,2004-02-26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1139251,38500,1026470,332,121,3.0,2001,3486.0,High,2011-05-19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1139253,11000,1057373,17311,121,3.0,2007,722.0,Medium,2009-07-23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401120,6333336,10500,1840702,21439,149,1.0,2005,NaN,NaN,2011-11-02,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
401121,6333337,11000,1830472,21439,149,1.0,2005,NaN,NaN,2011-11-02,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
401122,6333338,11500,1887659,21439,149,1.0,2005,NaN,NaN,2011-11-02,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
401123,6333341,9000,1903570,21435,149,2.0,2005,NaN,NaN,2011-10-25,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN


可以看到，行和列都显示不全，要解决这个问题，通过display_all函数封装了一下

In [52]:
display_all(df_raw.tail().T)

,401120,401121,401122,401123,401124
SalesID,6333336,6333337,6333338,6333341,6333342
SalePrice,9.259131,9.305651,9.350102,9.10498,8.955448
MachineID,1840702,1830472,1887659,1903570,1926965
ModelID,21439,21439,21439,21435,21435
datasource,149,149,149,149,149
auctioneerID,1.0,1.0,1.0,2.0,2.0
YearMade,2005,2005,2005,2005,2005
MachineHoursCurrentMeter,NaN,NaN,NaN,NaN,NaN
UsageBand,NaN,NaN,NaN,NaN,NaN
saledate,2011-11-02 00:00:00,2011-11-02 00:00:00,2011-11-02 00:00:00,2011-10-25 00:00:00,2011-10-25 00:00:00


<code>tail()</code> 显示最后五行。    
<code>DataFrame.T</code>   转置，由于列比较多，转置后显示

**再提醒一下，熟练使用 Shift+Tab**

In [46]:
display_all(df_raw.describe(include='all', datetime_is_numeric=True).T)

,count,unique,top,freq,mean,min,25%,50%,75%,max,std
SalesID,401125.0,NaN,NaN,NaN,1919712.521503,1139246.0,1418371.0,1639422.0,2242707.0,6333342.0,909021.492667
SalePrice,401125.0,NaN,NaN,NaN,31099.712848,4750.0,14500.0,24000.0,40000.0,142000.0,23036.898502
MachineID,401125.0,NaN,NaN,NaN,1217902.517971,0.0,1088697.0,1279490.0,1468067.0,2486330.0,440991.954249
ModelID,401125.0,NaN,NaN,NaN,6889.70298,28.0,3259.0,4604.0,8724.0,37198.0,6221.777842
datasource,401125.0,NaN,NaN,NaN,134.66581,121.0,132.0,132.0,136.0,172.0,8.962237
auctioneerID,380989.0,NaN,NaN,NaN,6.55604,0.0,1.0,2.0,4.0,99.0,16.976779
YearMade,401125.0,NaN,NaN,NaN,1899.156901,1000.0,1985.0,1995.0,2000.0,2013.0,291.797469
MachineHoursCurrentMeter,142765.0,NaN,NaN,NaN,3457.955353,0.0,0.0,0.0,3025.0,2483300.0,27590.256413
UsageBand,69639,3,Medium,33985,NaN,NaN,NaN,NaN,NaN,NaN,NaN
saledate,401125,NaN,NaN,NaN,2004-08-02 07:47:13.986413184,1989-01-17 00:00:00,2000-10-07 00:00:00,2006-03-23 00:00:00,2009-03-24 00:00:00,2011-12-30 00:00:00,NaN


<code>DataFram.describe</code> 显示每一列的统计信息，对数据有个大略的印象
>  FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.

In [47]:
df_raw.SalePrice = np.log(df_raw.SalePrice)

上面代码把SalePrice这一列的数据都求了一下log，可以显示出来和之前数据比较一下：

In [48]:
display(df_raw.SalePrice.tail())

401120    9.259131
401121    9.305651
401122    9.350102
401123    9.104980
401124    8.955448
Name: SalePrice, dtype: float64

因为算法结果的评价使用 RMSLE (root mean squared log error)，因此对SalePrice统一求个log，后续直接算RMSE(均方根误差)

$RMSE=\sqrt{\frac{1}{n}\sum_{i=1}^n (y_i - \hat{y_i})^2}$

$RMLSE=\sqrt{\frac{1}{n}\sum_{i=1}^n (log(y_i + 1) - log(\hat{y_i}+1))^2}$

> 网上抄的公式，不清楚为啥RMLSE算log的时候要用y+1

## 开始尝试跑算法

使用的是sklearn框架。

一般是这个套路：

> alg=xxx()    #初始化算法实例<p>
alg.fit(x,y)    #训练<p>
yy = alg.predict(xx)    #推理<p>


In [51]:
RandomForestRegressor

sklearn.ensemble._forest.RandomForestRegressor

这里要预测的是售价，因此x(independent variables)取SalePrice之外的所有数据，y(dependent variable)就是SalePrice

In [49]:
m = RandomForestRegressor(n_jobs=-1)
# The following code is supposed to fail due to string values in the input data
m.fit(df_raw.drop('SalePrice', axis=1), df_raw.SalePrice)

ValueError: could not convert string to float: 'Low'

出了错直接看最后一行： 不能正确处理string类型

rf算法需要把所有输入都转换成数字

首先是处理 **日期时间** 数据。

date time很有用，比如是不是周末？ 重大节假日？ 晚高峰？ 这些可能都对数据结果有很大影响

因此需要对dtype数据进行更精细的解析

In [83]:
add_datepart(df_raw, 'saledate')
df_raw.saleYear.head()

F:\fastai\fastai1\old\fastai0\structured.py:123: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  for n in attr: df[targ_pre + n] = getattr(fld.dt, n.lower())


0    2006
1    2004
2    2004
3    2011
4    2009
Name: saleYear, dtype: int64

In [60]:
df_raw.shape  #可以看到，增加了几列

(401125, 65)

In [84]:
df_raw.columns #删除了saledate列，最后增加若干列，将日期时间细分

Index(['SalesID', 'SalePrice', 'MachineID', 'ModelID', 'datasource',
       'auctioneerID', 'YearMade', 'MachineHoursCurrentMeter', 'UsageBand',
       'fiModelDesc', 'fiBaseModel', 'fiSecondaryDesc', 'fiModelSeries',
       'fiModelDescriptor', 'ProductSize', 'fiProductClassDesc', 'state',
       'ProductGroup', 'ProductGroupDesc', 'Drive_System', 'Enclosure',
       'Forks', 'Pad_Type', 'Ride_Control', 'Stick', 'Transmission',
       'Turbocharged', 'Blade_Extension', 'Blade_Width', 'Enclosure_Type',
       'Engine_Horsepower', 'Hydraulics', 'Pushblock', 'Ripper', 'Scarifier',
       'Tip_Control', 'Tire_Size', 'Coupler', 'Coupler_System',
       'Grouser_Tracks', 'Hydraulics_Flow', 'Track_Type',
       'Undercarriage_Pad_Width', 'Stick_Length', 'Thumb', 'Pattern_Changer',
       'Grouser_Type', 'Backhoe_Mounting', 'Blade_Type', 'Travel_Controls',
       'Differential_Type', 'Steering_Controls', 'saleYear', 'saleMonth',
       'saleWeek', 'saleDay', 'saleDayofweek', 'saleDayofyear',


In [86]:
train_cats(df_raw) #将string转化成category(类似与枚举)

In [88]:
display_all(df_raw.UsageBand.head())
display_all(df_raw.UsageBand.tail(1)) #注意输出中的dtype是category，而不是object

0       Low
1       Low
2      High
3      High
4    Medium
Name: UsageBand, dtype: category
Categories (3, object): ['High' < 'Low' < 'Medium']

401124    NaN
Name: UsageBand, dtype: category
Categories (3, object): ['High' < 'Low' < 'Medium']

**需要注意，训练集和测试集要用相同的枚举值。 可以参考`apply_cats`**

In [89]:
df_raw.UsageBand.cat.categories

Index(['High', 'Low', 'Medium'], dtype='object')

rf算法的输入需要是数字，还需要将category转换成number

In [90]:
df_raw.UsageBand = df_raw.UsageBand.cat.codes #将category转换成数字
display_all(df_raw.UsageBand.head())

0    1
1    1
2    0
3    0
4    2
Name: UsageBand, dtype: int8

In [91]:
display_all(df_raw.UsageBand.tail(1)) #对于无数据的单元(NaN)，其值转换成-1

401124   -1
Name: UsageBand, dtype: int8

还需要处理数据中的空值。

下面输出了每一列的空数据所占的比例

In [93]:
display_all(df_raw.isnull().sum().sort_index()/len(df_raw))

Backhoe_Mounting            0.803872
Blade_Extension             0.937129
Blade_Type                  0.800977
Blade_Width                 0.937129
Coupler                     0.466620
Coupler_System              0.891660
Differential_Type           0.826959
Drive_System                0.739829
Enclosure                   0.000810
Enclosure_Type              0.937129
Engine_Horsepower           0.937129
Forks                       0.521154
Grouser_Tracks              0.891899
Grouser_Type                0.752813
Hydraulics                  0.200823
Hydraulics_Flow             0.891899
MachineHoursCurrentMeter    0.644089
MachineID                   0.000000
ModelID                     0.000000
Pad_Type                    0.802720
Pattern_Changer             0.752651
ProductGroup                0.000000
ProductGroupDesc            0.000000
ProductSize                 0.525460
Pushblock                   0.937129
Ride_Control                0.629527
Ripper                      0.740388
S

## 到这里休息一下，然后我们从头再来一遍

### 1.读取原始数据

In [5]:
df_raw = pd.read_csv(f'{PATH}Train.csv', low_memory=False, parse_dates=["saledate"])

In [17]:
add_datepart(df_raw, 'saledate') #加工日期时间

F:\fastai\fastai1\old\fastai0\structured.py:123: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  for n in attr: df[targ_pre + n] = getattr(fld.dt, n.lower())


In [18]:
df_raw.SalePrice = np.log(df_raw.SalePrice) #y预先求log

### 2.保存数据

In [19]:
os.makedirs('tmp', exist_ok=True)
df_raw.to_feather('tmp/bulldozers-raw') #Write a DataFrame to the binary Feather format.

In the future we can simply read it from this fast format.

In [7]:
df_raw = pd.read_feather('tmp/bulldozers-raw')

### 3.数据预处理

`proc_df`把我们前面尝试的几个事都做了：
- 字符串转换成数字
- 对于不完整的数据补充初值
- 分别提取 X 和 y

In [20]:
df, y, nas = proc_df(df_raw, 'SalePrice')

In [21]:
display_all(df.tail())

NameError: name 'display_all' is not defined

In [22]:
y.shape

(401125,)

In [15]:
y

array([66000, 57000, 10000, ..., 11500,  9000,  7750], dtype=int64)

In [16]:
nas

{'auctioneerID': 2.0, 'MachineHoursCurrentMeter': 0.0}

这里多说几句数据补充  
对于连续数据，如果某列中有miss的数据(NaN), 增加一列`列名_na`。对于正常的数据行，新增列中填false，对于空的数据行，新增列中填True；空数据填此列平均值。 

对于离散的数据，在前面字符串处理中(转成category)已做了处理，NaN的取值为-1

### 4.跑算法

In [101]:
m = RandomForestRegressor(n_jobs=-1)
m.fit(df, y)
m.score(df,y)

0.987819260639956

可以看到训练的效果还是很好的    
虽然什么原理都不懂，但是我们能开车了 @_@  

当然这个结果没什么意义，因为我们用同一组数据进行训练和推理，可能会过拟合。

可以通过下面的方法，把数据分为训练集(train)和验证集(validation)

In [10]:
def split_vals(a,n): return a[:n].copy(), a[n:].copy()

n_valid = 12000  # same as Kaggle's test set size
n_trn = len(df)-n_valid
raw_train, raw_valid = split_vals(df_raw, n_trn)
X_train, X_valid = split_vals(df, n_trn)
y_train, y_valid = split_vals(y, n_trn)

X_train.shape, y_train.shape, X_valid.shape

((389125, 54), (389125,), (12000, 54))

### 5.这段结束，消化一下再继续